<a href="https://colab.research.google.com/github/arghac14/Customer-Churn-Analysis/blob/master/Ensemble%20Learning/Voting%20Classifier/VotingClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/SohelRaja/ML_repo_college/master/Decision%20Tree%20with%20Python/WA_Fn-UseC_-Telco-Customer-Churn.csv'
telco = pd.read_csv(url)
telco=telco.drop(['customerID','TotalCharges'] , axis=1)
telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,Yes


In [3]:
telco['Partner'].replace(to_replace=[['No'],['Yes']],value=['PartnerNo','PartnerYes'],inplace=True)
telco['Dependents'].replace(to_replace=[['No'],['Yes']],value=['Independent','Dependent'],inplace=True)
telco['PhoneService'].replace(to_replace=[['No'],['Yes']],value=['PSNo','PSYes'],inplace=True)
telco['PaperlessBilling'].replace(to_replace=[['No'],['Yes']],value=['PLBNo','PLBYes'],inplace=True)
telco['MultipleLines'].replace(to_replace=[['No phone service'],['No'],['Yes']],value=['MLNo','MLNo','MLYes'],inplace=True)
telco['OnlineSecurity'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['OSNo','OSNo','OSYes'],inplace=True)
telco['OnlineBackup'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['OBNo','OBNo','OBYes'],inplace=True)
telco['DeviceProtection'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['DPNo','DPNo','DPYes'],inplace=True)
telco['TechSupport'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['TSNo','TSNo','TSYes'],inplace=True)
telco['StreamingTV'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['STVNo','STVNo','STVYes'],inplace=True)
telco['StreamingMovies'].replace(to_replace=[['No internet service'],['No'],['Yes']],value=['SMNo','SMNo','SMYes'],inplace=True)
telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,Female,0,PartnerYes,Independent,1,PSNo,MLNo,DSL,OSNo,OBYes,DPNo,TSNo,STVNo,SMNo,Month-to-month,PLBYes,Electronic check,29.85,No
1,Male,0,PartnerNo,Independent,34,PSYes,MLNo,DSL,OSYes,OBNo,DPYes,TSNo,STVNo,SMNo,One year,PLBNo,Mailed check,56.95,No
2,Male,0,PartnerNo,Independent,2,PSYes,MLNo,DSL,OSYes,OBYes,DPNo,TSNo,STVNo,SMNo,Month-to-month,PLBYes,Mailed check,53.85,Yes
3,Male,0,PartnerNo,Independent,45,PSNo,MLNo,DSL,OSYes,OBNo,DPYes,TSYes,STVNo,SMNo,One year,PLBNo,Bank transfer (automatic),42.30,No
4,Female,0,PartnerNo,Independent,2,PSYes,MLNo,Fiber optic,OSNo,OBNo,DPNo,TSNo,STVNo,SMNo,Month-to-month,PLBYes,Electronic check,70.70,Yes


In [4]:
telco['gender'].replace(to_replace=[['Female'],['Male']],value=[0,1],inplace=True)
telco['Partner'].replace(to_replace=[['PartnerNo'],['PartnerYes']],value=[0,1],inplace=True)
telco['Dependents'].replace(to_replace=[['Independent'],['Dependent']],value=[0,1],inplace=True)
telco['PhoneService'].replace(to_replace=[['PSNo'],['PSYes']],value=[0,1],inplace=True)
telco['PaperlessBilling'].replace(to_replace=[['PLBNo'],['PLBYes']],value=[0,1],inplace=True)
telco['MultipleLines'].replace(to_replace=[['MLNo'],['MLYes']],value=[0,1],inplace=True)
telco['OnlineSecurity'].replace(to_replace=[['OSNo'],['OSYes']],value=[0,1],inplace=True)
telco['OnlineBackup'].replace(to_replace=[['OBNo'],['OBYes']],value=[0,1],inplace=True)
telco['DeviceProtection'].replace(to_replace=[['DPNo'],['DPYes']],value=[0,1],inplace=True)
telco['TechSupport'].replace(to_replace=[['TSNo'],['TSYes']],value=[0,1],inplace=True)
telco['StreamingTV'].replace(to_replace=[['STVNo'],['STVYes']],value=[0,1],inplace=True)
telco['StreamingMovies'].replace(to_replace=[['SMNo'],['SMYes']],value=[0,1],inplace=True)
telco['InternetService'].replace(to_replace=[['DSL'],['Fiber optic'],['No']],value=[0,1,2],inplace=True)
telco['PaymentMethod'].replace(to_replace=[['Electronic check'],['Mailed check'],['Bank transfer (automatic)'],['Credit card (automatic)']],value=[0,1,2,3],inplace=True)
telco['Contract'].replace(to_replace=[['Month-to-month'], ['One year'], ['Two year']],value=[0,1,2],inplace=True)
telco['Churn'].replace(to_replace=[['No'], ['Yes']],value=[0,1],inplace=True)
telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,29.85,0
1,1,0,0,0,34,1,0,0,1,0,1,0,0,0,1,0,1,56.95,0
2,1,0,0,0,2,1,0,0,1,1,0,0,0,0,0,1,1,53.85,1
3,1,0,0,0,45,0,0,0,1,0,1,1,0,0,1,0,2,42.30,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,0,70.70,1


In [5]:
telco.describe()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.421837,0.872923,0.286668,0.344881,0.343888,0.290217,0.384353,0.387903,0.690473,0.592219,1.315633,64.761692,0.265370
std,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.493888,0.737796,0.452237,0.475363,0.475038,0.453895,0.486477,0.487307,0.833755,0.491457,1.148907,30.090047,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.500000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,70.350000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,89.850000,1.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,118.750000,1.000000


In [6]:
df=telco
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,29.85,0
1,1,0,0,0,34,1,0,0,1,0,1,0,0,0,1,0,1,56.95,0
2,1,0,0,0,2,1,0,0,1,1,0,0,0,0,0,1,1,53.85,1
3,1,0,0,0,45,0,0,0,1,0,1,1,0,0,1,0,2,42.30,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,0,70.70,1


## Applying classifier models

In [7]:
from sklearn.model_selection import train_test_split
import math

X=telco[['gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges']]
X1=telco[['tenure','InternetService','PhoneService']]
y=telco['Churn']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=2)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)
X1_train,X1_test,y_train,y_test=train_test_split(X1,y,test_size=0.1,random_state=2)

Train set: (6338, 18) (6338,)
Test set: (705, 18) (705,)


### logistic Regression:

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

reg=LogisticRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)

lr_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('\nAccuracy(Logistic Regression): ',lr_accuracy,'%\n')


Accuracy(Logistic Regression):  80.57 %



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
# With Best Features:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

reg=LogisticRegression()
reg.fit(X1_train,y_train)
y_pred=reg.predict(X1_test)

lr_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('\nAccuracy(Logistic Regression): ',lr_accuracy,'%\n')


Accuracy(Logistic Regression):  78.72 %



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### KNN Classifier:

In [0]:
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [11]:
K=100
acc=np.zeros((K-1))
for k in range(1,K):
  model=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
  y_pred=model.predict(X_test)
  acc[k-1]=metrics.accuracy_score(y_test,y_pred)
#print (acc)
knn_accuracy=round(acc.max()*100,2)
print("The Best Accuracy is:",acc.max()*100,"% with k=",acc.argmax()+1)

The Best Accuracy is: 81.84397163120568 % with k= 84


In [12]:
# With Best features

K=100
acc=np.zeros((K-1))
for k in range(1,K):
  model=KNeighborsClassifier(n_neighbors=k).fit(X1_train,y_train)
  y_pred=model.predict(X1_test)
  acc[k-1]=metrics.accuracy_score(y_test,y_pred)
#print (acc)
knn_accuracy=round(acc.max()*100,2)
print("The Best Accuracy is:",acc.max()*100,"% with k=",acc.argmax()+1)

The Best Accuracy is: 82.41134751773049 % with k= 38


### Naive-Bayes Classifier

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
clf=GaussianNB()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

nb_accuracy=round(metrics.accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Naive Bayes): ',nb_accuracy,'%')

Accuracy(Naive Bayes):  76.31 %


In [14]:
# With Best Features

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
clf=GaussianNB()
clf.fit(X1_train,y_train)
y_pred=clf.predict(X1_test)

nb_accuracy=round(metrics.accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Naive Bayes): ',nb_accuracy,'%')


Accuracy(Naive Bayes):  80.71 %


### Decision Tree Classifier

In [15]:
from sklearn.tree import DecisionTreeClassifier

clf_entropy=DecisionTreeClassifier(criterion='entropy', random_state=1)
clf_entropy.fit(X_train,y_train)
y_pred=clf_entropy.predict(X_test)

dt_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Decision Tree): ',dt_accuracy,'%')

Accuracy(Decision Tree):  74.61 %


In [16]:
# With Best Features

from sklearn.tree import DecisionTreeClassifier

clf_entropy=DecisionTreeClassifier(criterion='entropy', random_state=1)
clf_entropy.fit(X1_train,y_train)
y_pred=clf_entropy.predict(X1_test)

dt_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Decision Tree): ',dt_accuracy,'%')

Accuracy(Decision Tree):  81.99 %


### Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

rf_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Random Forest): ',rf_accuracy,'%')

Accuracy(Random Forest):  80.0 %


In [18]:
# With Best Featuers

from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=1000)

clf.fit(X1_train,y_train)

y_pred=clf.predict(X1_test)

rf_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(Random Forest): ',rf_accuracy,'%')

Accuracy(Random Forest):  82.41 %


### Support Vector Classifier

In [19]:
from sklearn import svm

clf=svm.SVC()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

svm_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(SVM): ',svm_accuracy,'%')

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy(SVM):  81.28 %


In [20]:
# With Best Features


from sklearn import svm

clf=svm.SVC()
clf.fit(X1_train,y_train)
y_pred=clf.predict(X1_test)

svm_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('Accuracy(SVM): ',svm_accuracy,'%')

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy(SVM):  82.13 %


# Ensemble Learning:
## Voting Classifier:

In [21]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(kernel="linear", C=1000)
nb_clf=GaussianNB()
dt_clf=DecisionTreeClassifier(criterion='entropy', random_state=2)
knn_clf=KNeighborsClassifier(n_neighbors=38)

voting_clf = VotingClassifier(
	estimators = [('lr', log_clf), ('rf', rnd_clf), ('knn',knn_clf),('dt',dt_clf),('nb', nb_clf)])    
voting_clf.fit(X_train, y_train)
y_pred=voting_clf.predict(X_test)

vot_accuracy1=round(accuracy_score(y_test,y_pred)*100,2)
print('\n\nAccuracy(Voting Classifier): ',vot_accuracy1,'%')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)




Accuracy(Voting Classifier):  80.71 %


In [22]:
# With Best Features

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier(n_estimators=1000)
svm_clf = SVC()
nb_clf=GaussianNB()
dt_clf=DecisionTreeClassifier(criterion='entropy', random_state=1)
knn_clf=KNeighborsClassifier(n_neighbors=38)

voting_clf = VotingClassifier(
	estimators = [('lr', log_clf), ('rf', rnd_clf),('svm',svm_clf), ('knn',knn_clf),('dt',dt_clf),('nb', nb_clf)])    
voting_clf.fit(X1_train, y_train)
y_pred=voting_clf.predict(X1_test)

vot_accuracy=round(accuracy_score(y_test,y_pred)*100,2)
print('\n\nAccuracy(Voting Classifier): ',vot_accuracy,'%')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)




Accuracy(Voting Classifier):  82.41 %


# Accuracy report:


In [23]:
print('-----------Accuracy of prediction--------------')
# print('\nLogistic Regression : ',lr_accuracy,'%')
# print('\nK-nearest Neighbors : ',knn_accuracy,'%')
# print('\nNaive Bayes : ',nb_accuracy,'%')
# print('\nDecision Tree : ',dt_accuracy,'%')
# print('\nRandom Forest : ',rf_accuracy,'%')
# print('\nSupport Vector Machine : ',svm_accuracy,'%')
# print('\n\nFinal Accuracy-')
# print('\nVoting Classifier: ',vot_accuracy,'%')

accu=pd.DataFrame({
    'Model':['Logistic Regression','K-nearest Neighbors','Support Vector Machine','Naive Bayes','Decision Tree','Random Forest','Voting Classifier(Final Accuracy)'],
    'Accuracy':[lr_accuracy,knn_accuracy,svm_accuracy,nb_accuracy,dt_accuracy,rf_accuracy,vot_accuracy]
})

print("\nAccuracy with all features: ",vot_accuracy1,"%","\nAccuracy with best features: ",vot_accuracy,"%\n")
print("\t-------Ensemble Learning------\n")
accu

-----------Accuracy of prediction--------------

Accuracy with all features:  80.71 % 
Accuracy with best features:  82.41 %

	-------Ensemble Learning------



,Model,Accuracy
0,Logistic Regression,78.72
1,K-nearest Neighbors,82.41
2,Support Vector Machine,82.13
3,Naive Bayes,80.71
4,Decision Tree,81.99
5,Random Forest,82.41
6,Voting Classifier(Final Accuracy),82.41
